# Bayesian Mixture Model

This notebook illustrate how to build and train a Bayesian Mixture Model with the [beer framework](https://github.com/beer-asr/beer).

In [1]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.insert(0, '../')

import copy

import beer
import numpy as np
import torch

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
from bokeh.models import LinearAxis, Range1d
output_notebook()

# Convenience functions for plotting.
import plotting

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

## Data

As an illustration, we generate a synthetic data set composed of two Normal distributed cluster. One has a diagonal covariance matrix whereas the other has a dense covariance matrix.

In [2]:
# First cluster.
mean = np.array([-1.5, 4]) 
cov = np.array([[.75, 0], [0, 2.]])
data1 = np.random.multivariate_normal(mean, cov, size=200)

# Second cluster.
mean = np.array([5, 5]) 
cov = np.array([[2, 1], [1, .75]])
data2 = np.random.multivariate_normal(mean, cov, size=200)

# Merge everything to get the finale data set.
data = np.vstack([data1, data2])
np.random.shuffle(data)

In [3]:
# Mean, variance of the data to scale the figure.
mean = data.mean(axis=0)
var = data.var(axis=0)
std_dev = np.sqrt(max(var))
x_range = (mean[0] - 2 * std_dev, mean[0] + 2 * std_dev)
y_range = (mean[1] - 2 * std_dev, mean[1] + 2 * std_dev)
global_range = (min(x_range[0], y_range[0]), max(x_range[1], y_range[1]))

fig = figure(title='Data', width=400, height=400,
             x_range=global_range, y_range=global_range)
fig.circle(data[:, 0], data[:, 1])

show(fig)

## Model Creation

We create two types of mixture model: one whose (Normal) components have full covariance matrix and the other whose (Normal) components have diagonal covariance matrix.

In [4]:
ncomps = 10 

# We use the global mean/cov. matrix of the data to initialize the mixture.
p_mean = torch.from_numpy(data.mean(axis=0)).float()
p_cov = torch.from_numpy(np.cov(data.T)).float()

# GMM (diag cov).
prior_weights = beer.DirichletPrior(torch.ones(ncomps) * 1e-1)
posterior_weights = beer.DirichletPrior(torch.ones(ncomps))
prior = beer.NormalGammaPrior(p_mean, torch.ones(2), 1.)
posts = [beer.NormalGammaPrior(p_mean + .1 * torch.randn(2), torch.ones(2), 1e-1)
         for _ in range(ncomps)]
normalset = beer.NormalDiagonalCovarianceSet(prior, posts)
gmm_diag = beer.Mixture(prior_weights, posterior_weights, normalset)

# GMM (full cov).
prior_weights = beer.DirichletPrior(torch.ones(ncomps) * 1)
posterior_weights = beer.DirichletPrior(torch.ones(ncomps))
prior = beer.NormalWishartPrior(p_mean, torch.eye(2), 1.)
posts = [beer.NormalWishartPrior(p_mean + .1 * torch.randn(2), torch.eye(2), 1)
         for _ in range(ncomps)]
normalset = beer.NormalFullCovarianceSet(prior, posts)
gmm_full = beer.Mixture(prior_weights, posterior_weights, normalset)

# GMM shared (full) cov.
prior_weights = beer.DirichletPrior(torch.ones(ncomps) * 1)
posterior_weights = beer.DirichletPrior(torch.ones(ncomps))
p_means = torch.cat([p_mean[None]]*ncomps, dim=0)
prior = beer.JointNormalGammaPrior(p_means, torch.ones(2), 1)
posterior = beer.JointNormalGammaPrior(p_means + .1 * torch.randn(ncomps, 2), 
    torch.ones(2), 1e-1)
normalset = beer.NormalSetSharedDiagonalCovariance(prior, posterior, ncomps)
gmm_sharedcov_diag = beer.Mixture(prior_weights, posterior_weights, normalset)

# GMM shared (full) cov.
prior_weights = beer.DirichletPrior(torch.ones(ncomps) * 1)
posterior_weights = beer.DirichletPrior(torch.ones(ncomps))
p_means = torch.cat([p_mean[None]]*ncomps, dim=0)
prior = beer.JointNormalWishartPrior(p_means, torch.eye(2), 1)
posterior = beer.JointNormalWishartPrior(p_means + .1 * torch.randn(ncomps, 2), 
    torch.eye(2), 1e-1)
normalset = beer.NormalSetSharedFullCovariance(prior, posterior, ncomps)
gmm_sharedcov_full = beer.Mixture(prior_weights, posterior_weights, normalset)

## Variational Bayes Training 

In [5]:
epochs = 200
lrate = 1.
X = torch.from_numpy(data).float()
loss_fn = beer.StochasticVariationalBayesLoss(len(X))
params = gmm_diag.parameters + gmm_full.parameters 
params += gmm_sharedcov_full.parameters
params += gmm_sharedcov_diag.parameters
optimizer = beer.BayesianModelOptimizer(params, lrate)
    
elbos_diag, elbos_full, elbos_sharedcov_full, elbos_sharedcov_diag = [], [], [], []
for epoch in range(epochs):
    optimizer.zero_grad()
    loss_diag = loss_fn(gmm_diag, X)
    loss_diag.backward()
    loss_full = loss_fn(gmm_full, X)
    loss_full.backward()
    loss_sharedcov_diag = loss_fn(gmm_sharedcov_diag, X)
    loss_sharedcov_diag.backward()
    loss_sharedcov_full = loss_fn(gmm_sharedcov_full, X)
    loss_sharedcov_full.backward()
    optimizer.step()
    
    if epoch > 0:
        elbos_diag.append(float(loss_diag) / len(X))
        elbos_full.append(float(loss_full) / len(X))
        elbos_sharedcov_diag.append(float(loss_sharedcov_diag) / len(X))
        elbos_sharedcov_full.append(float(loss_sharedcov_full) / len(X))

# Plot the ELBO.
fig = figure(title='ELBO', width=400, height=400, x_axis_label='step',
              y_axis_label='ln p(X)')
fig.line(np.arange(len(elbos_diag)), elbos_diag, legend='GMM (diag)', color='blue')
fig.line(np.arange(len(elbos_full)), elbos_full, legend='GMM (full)', color='red')
fig.line(np.arange(len(elbos_sharedcov_diag)), elbos_sharedcov_diag, 
    legend='GMM (shared cov. diag)', color='green')
fig.line(np.arange(len(elbos_sharedcov_full)), elbos_sharedcov_full, 
    legend='GMM (shared cov. full)', color='black')
fig.legend.location = 'bottom_right'

show(fig)

In [6]:
fig1 = figure(title='GMM (diag)', x_range=global_range, y_range=global_range,
              width=400, height=400)
fig1.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig1, gmm_diag, color='blue')

fig2 = figure(title='GMM (full)', x_range=global_range, y_range=global_range,
              width=400, height=400)
fig2.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig2, gmm_full, color='red')

fig3 = figure(title='GMM (shared cov. full)', x_range=global_range, y_range=global_range,
              width=400, height=400)
fig3.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig3, gmm_sharedcov_diag, color='green')

fig4 = figure(title='GMM (shared cov. full)', x_range=global_range, y_range=global_range,
              width=400, height=400)
fig4.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig4, gmm_sharedcov_full, color='black')

grid = gridplot([[fig1, fig2], [fig3, fig4]])
show(grid)